In [11]:
!pip3 install google-genai databench-eval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.8 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64"

In [2]:
import pandas as pd
from google import genai

client = genai.Client(api_key="")

qa_df = pd.read_csv('/kaggle/input/semvaltask8/competition/test_qa.csv')
base_data_path = '/kaggle/input/semvaltask8/competition/'

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:623: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [3]:
import textwrap
import numpy as np 

def postprocess_response(response: str, provided_dataset: str) -> str:
    code_from_llm = response.strip()
    if code_from_llm.startswith("```python"):
        code_from_llm = code_from_llm[len("```python"):].strip()
    if code_from_llm.startswith("```"):
         code_from_llm = code_from_llm[len("```"):].strip()
    if code_from_llm.endswith("```"):
        code_from_llm = code_from_llm[:-len("```")].strip()


    indented_code = textwrap.indent(code_from_llm, '    ')

    provided_df = pd.read_parquet(base_data_path + provided_dataset + '/sample.parquet') 
    func_code_string = code_from_llm
    
    try:

        exec_scope = {'pd': pd, 'np': np, 'df': provided_df }
        
    except Exception as e:
        print('data load error')
        return f"__DATA_LOAD_ERROR__: {e}"

    try:
        exec(func_code_string, exec_scope) # run the generated code with necessary scopes

        answer_func = exec_scope.get('generated_answer_func')
        if callable(answer_func):
            try:
                final_answer = answer_func(exec_scope['df'])
                
                if isinstance(final_answer, (pd.Series, pd.DataFrame)):
                    if isinstance(final_answer, pd.Series):
                        formatted_answer = str(final_answer.tolist())
                    else:
                         formatted_answer = "__CODE_ERROR__: Returned DataFrame, expected scalar/list/bool"
                elif isinstance(final_answer, list):
                    formatted_answer = str(final_answer)
                elif isinstance(final_answer, (int, float, bool, str, np.generic)):
                     formatted_answer = str(final_answer)
                elif final_answer is None:
                     formatted_answer = "__CODE_ERROR__: Returned None"
                else:
                    try:
                        formatted_answer = str(final_answer)
                    except Exception as fmt_e:
                         formatted_answer = f"__FORMAT_ERROR__: Could not convert {type(final_answer)} to string: {fmt_e}"

                if isinstance(formatted_answer, str) and "\n" in formatted_answer:
                    formatted_answer = formatted_answer.split("\n")[0]
                print('ans: ', formatted_answer)
                return formatted_answer

            except Exception as exec_e:
                print('exec error', exec_e)
                return f"__CODE_EXEC_ERROR__: {exec_e}"
        else:
            print('not callable')
            return "__CODE_DEF_ERROR__: generated_answer_func not callable"

    except SyntaxError as syn_e:
         return f"__CODE_SYNTAX_ERROR__: {syn_e}"
    except Exception as e:
        return f"__EXEC_ERROR__: {e}"

In [29]:
def generate_prompt_messages(row: dict) -> list:
    question = row["question"]
    dataset_name = row["dataset"]

    try:
        df_sample = pd.read_parquet(base_data_path + dataset_name + '/sample.parquet') 
        columns = df_sample.columns.tolist()
        dtypes = df_sample.dtypes.to_dict()
        head_csv = df_sample.head(5).to_csv(index=False)
        del df_sample 
    except Exception as e:
        print(f"Warning: Could not load sample for {dataset_name} for prompt generation: {e}")
        columns = ["unknown"]
        dtypes = {}
        head_csv = "Error loading sample data"

    messages = f"""Question: {question}
# Table schema
Dataset columns: {columns}
Data Types: {dtypes}
# Output format:
Boolean - if asking a yes-no, true-false question
Cell Value - if the answer should be a value from dataframe
Number - if the answer should be a numerical computed value or just a numerical value from dataframe
List[cell values] - if asked for a list of values from cells of the dataframe
List[number] - if asked for a list of numerical values either computed or directly from cells
# Rules:
You have access to pandas and numpy only
Be careful about types and do type conversion when necessary
Don't return any explanation, formatting or comment
You should write short, concise and efficient code
Pay attention to return type

Now, write the function below with proper indentation to return the answer to the question.

def generated_answer_func(df: pd.DataFrame):
"""
    return messages

In [28]:
from google.genai import types
from time import sleep

def gemini_model_call(prompt_messages_batch: list) -> list:
    results = []
    for messages in prompt_messages_batch:
        try:
            response = client.models.generate_content(
                model="gemini-2.0-flash", 
                config=types.GenerateContentConfig(
                    system_instruction="You are a python programmer who takes a pandas dataset, a question and returns the code to answer that question based on solely the dataset."),
                contents=messages
            )
            results.append(response.text)
            sleep(5)
            
        except Exception as e:
            print(f"Error during model generation: {e}")
            results.append(f"__CODE_GEN_ERROR__: {e}")

    return results

In [27]:
from datasets import Dataset 
from databench_eval import Runner, Evaluator

qa = pd.read_csv('/kaggle/input/semvaltask8/competition/test_qa.csv') # Use 'test' split for test_qa.csv
qa_dataset = Dataset.from_pandas(qa) # Convert to Hugging Face Dataset
print(f"Loaded {len(qa_dataset)} questions.")

runner_lite = Runner(
    model_call=gemini_model_call,
    prompt_generator=generate_prompt_messages, # Function returning messages list
    postprocess=lambda response, dataset: postprocess_response(response, dataset),
    qa=qa_dataset, # Pass the HF Dataset object
    batch_size=6,  # Adjust based on GPU memory and desired speed vs stability
                    # Note: True batching not implemented in llama3_model_call above
)


Loaded 522 questions.


In [30]:


print("--- Running Lite Predictions ---")
responses_lite_path = "predictions_lite.txt"
responses_lite = runner_lite.run(save=responses_lite_path)
print(f"Lite predictions saved to {responses_lite_path}")



print("\n--- Evaluating ---")

if os.path.exists(responses_lite_path):
    accuracy_lite = evaluator.eval(responses_lite, lite=True)
    print(f"DataBench (Lite) Accuracy: {accuracy_lite*100:.2f}%")
else:
    print(f"ERROR: {responses_lite_path} not found. Cannot evaluate lite dataset.")

--- Running Lite Predictions ---


  1%|          | 1/87 [00:35<50:22, 35.15s/it]

ans:  True
ans:  True
ans:  False
ans:  False
ans:  True
ans:  True


<string>:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  2%|▏         | 2/87 [01:10<49:54, 35.23s/it]

ans:  True
ans:  True
ans:  True
ans:  Laboratory Technician
ans:  Sales
ans:  Divorced
Error during model generation: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}


...

 99%|█████████▉| 86/87 [48:39<00:34, 34.38s/it]

ans:  [492]
ans:  [4.0, 6.0, 21.0, 38.0, 18.0]
ans:  [11.0, 23.0, 27.0, 10.0, 13.0]
ans:  [498]
ans:  Elon Musk
ans:  ['History and Tradition', ' Business, Investment and Economics', 'Islamic Books', 'Islamic Books', 'Computer Science & Engineering']


100%|██████████| 87/87 [49:13<00:00, 33.95s/it]

ans:  ['Madinah Arabic Reader 1', 'How Harsh Mariwala ?Groomed? Marico', 'Train to Pakistan']
ans:  ['History and Tradition', ' Business, Investment and Economics', 'Novel']
ans:  ['Yuval Noah Harari', 'Benjamin Graham', 'Khaled Hosseini']
ans:  ['8th Impression']
Lite predictions saved to predictions_lite.txt

--- Evaluating ---


NameError: name 'os' is not defined

**Evaluation:** 74.3% accuracy